# Local LLM LangChain v1 Agent

This notebook demonstrates how to wire a LangChain v1 agent to a locally hosted LLM (default: [Ollama](https://ollama.ai)).


## Prerequisites

- Start your local model server (e.g., run `ollama serve` after pulling a model such as `ollama pull llama3`).
- Install the project dependencies with `uv sync` inside a virtual environment (see README).
- Create a `.env` file (see `.env.example`) with `MODEL_NAME` and `OLLAMA_BASE_URL`.


In [ ]:
# Optional: install dependencies via uv from the notebook (requires uv on PATH)
# !uv sync


In [ ]:
import os
import datetime as dt
import platform
from typing import Any

from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_ollama import ChatOllama


In [ ]:
# Load env vars and define sane defaults for local development
load_dotenv()
MODEL_NAME = os.getenv("MODEL_NAME", "llama3")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
TEMPERATURE = float(os.getenv("MODEL_TEMPERATURE", "0.2"))
print(f"Using model={MODEL_NAME} @ {OLLAMA_BASE_URL}")


In [ ]:
# Instantiate the local LLM client (Ollama by default)
llm = ChatOllama(
    base_url=OLLAMA_BASE_URL,
    model=MODEL_NAME,
    temperature=TEMPERATURE,
)


In [ ]:
@tool
def system_context(request: str = "") -> str:
    """Return contextual information about the workstation (OS, time)."""
    now = dt.datetime.now().isoformat(timespec="seconds")
    return f"System time: {now}\nPlatform: {platform.platform()}"


@tool
def python_math(expression: str) -> str:
    """Safely evaluate simple math expressions using Python syntax."""
    allowed_globals: dict[str, Any] = {"__builtins__": {}}
    try:
        result = eval(expression, allowed_globals, {})
    except Exception as exc:  # noqa: BLE001 - provide feedback to agent
        return f"error: {exc}"
    return str(result)


tools = [system_context, python_math]
for t in tools:
    print(f"Loaded tool: {t.name}")


In [ ]:
SYSTEM_PROMPT = os.getenv(
    "SYSTEM_PROMPT",
    "You are a helpful local-only AI agent. Use the available tools when they can improve the answer before responding.",
)
print("Using LangChain 1.x tool-calling agent graph (model must support native tool calls).")
agent_runnable = create_agent(
    model=llm,
    tools=tools,
    system_prompt=SYSTEM_PROMPT,
)

def invoke_agent(user_input: str) -> str:
    try:
        state = agent_runnable.invoke({"messages": [{"role": "user", "content": user_input}]})
    except Exception as exc:
        message = str(exc).lower()
        if "does not support tools" in message:
            raise RuntimeError("The current model/runtime does not expose tool calling. Switch to a tool-capable Ollama model (e.g., llama3.1, mistral-function) or rebuild with tool support.") from exc
        raise
    return state["messages"][-1].content


## Run the agent

Call `invoke_agent(...)` with a user prompt. Make sure your Ollama/LLM runtime exposes tool calling; otherwise create_agent() will raise.


In [ ]:
query = "What time is it?"
invoke_agent(query)
